In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from preprocessing.candlestick import PlotChart, get_min_time_unit

from tqdm import tqdm
import os
import datetime
import libs.utilities as utilities 

In [ ]:
fifteen_minutes_df = pd.read_csv('../../data/15m_ohlcv.csv', index_col=0)  
thirty_minutes_df = pd.read_csv('../../data/30m_ohlcv.csv', index_col=0)
two_hours_df = pd.read_csv('../../data/2h_ohlcv.csv', index_col=0)

fund_rate_df = pd.read_csv('../../data/Funding_Rate.csv', index_col=0)

In [ ]:
def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

### plot 3 dataframes in 2 rows

In [ ]:
dataframes_half = {'2h':two_hours_df.copy(), '30m':thirty_minutes_df.copy()}
dataframes_all = {'15m':fifteen_minutes_df.copy()}
dataframes = {**dataframes_half, **dataframes_all}

columns = ['Open', 'High', 'Low', 'Close']
periods = [5, 20, 50]
MAX_PRICE = 100000

for period in periods:
    name = 'MA_' + str(period)  
    columns.append(name)
    for df in dataframes.values():
        df[name] = df['Close'].rolling(period).mean()

min_unit = get_min_time_unit(list(dataframes_all.keys()))
df_columns = dataframes_all[min_unit].columns

rolled_column_name = np.roll(df_columns, -1)
for df in dataframes.values():
    df = df[rolled_column_name]

dataframes_all[min_unit]['Scaled'] = dataframes_all[min_unit][['Close']] / MAX_PRICE
plot_chart = PlotChart(dataframes_half, dataframes_all, dataframes, fund_rate_df, min_unit, periods, columns)

In [ ]:
def plot_Candel_MA(index, gauge=False, days=1010):
    current_index = index
    step = utilities.HYPERPARAMS['image_per_day']
    path = '../../images/candlesticks/'
    make_path(path)

    start = 0
    end = days * step

    for idx in tqdm(range(current_index - start, current_index - end, -1)):
        canvas = Image.new('RGB',size=(224,224))    
        image, date_time, price = plot_chart.plot_candel_image_df3(canvas, idx, plot_gauge=gauge)
        image.save(path + date_time.replace(' ', '_').replace(':', '-') + '_' + price + '.jpg')

In [ ]:
current_index = len(dataframes_all[min_unit].iloc[dataframes_all[min_unit].index < '2022-09-01 00:00:00']) - 1
plot_Candel_MA(current_index, gauge=True) #2019-11-25 ~ 2022-08-31